## How to use Itinerary Builder

Itinerary Builder is a module used to query a database of possible itineraries, and return a dataframe with information on each. The critical outputs include the origins and destinations of each two flight itinerary, their times, airlines, and durations. Included as well is the next best second leg flight for each if the connecting flight is missed.

A machine learning algorithm is used to weight the time cost of a missed connection based on the liklihood of the missed connnection occuring.

Users of the module must provide a connection time assumption. This is the minimum time between connecting flights which the user finds acceptable (e.g., allow 45 minutes minimum for connection).

Examples of each function's usage is below.

In [1]:
from itineraryBuilder import *

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Itinerary Builder main function call

With default options, the resulting dataframe is ordered by risk as shown below. Note that "arr_no_later_date" is restricting the results such that itineraries which arrive at their destination the following day are ommitted. This occurs if this parameter equals "flight_date".

In [3]:
origin = 'ATL'
destination = 'SEA'
flight_date = '12/3/2019'

dne = '700'
dnl = '1000'
ane = '1'
anl = '2000'
tc = 45
arr_no_later_date = '12/3/2019'

df = itineraryBuilder('faa_2019_12', origin, destination, flight_date, arr_no_later_date, tc, dne, dnl, ane, anl); df.head(10)

C:\Users\parso\OneDrive\Documents\Education\Data Analytics\CSE-6242_Data and Visual Analytics\Project\gatech-cse6242-spring2023-project\dataManagement\itineraryBuilder.py:108: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  df['FIRST_LEG_ARR_TIMESTAMP'] = df['FIRST_LEG_ARR_TIMESTAMP'] + df['overnight_bool_1'].astype('timedelta64[D]')


,FIRST_LEG_AIRLINE,FIRST_LEG_ORIG,FIRST_LEG_ORIG_CITY,FIRST_LEG_DEST,FIRST_LEG_DEST_CITY,FIRST_LEG_DATE,FIRST_LEG_DEP_TIME,FIRST_LEG_ARR_TIME,FIRST_LEG_PRED15,FIRST_LEG_PRED30,FIRST_LEG_PRED45,FIRST_LEG_PRED60,FIRST_LEG_PRED75,FIRST_LEG_PRED90,FIRST_LEG_PRED105,FIRST_LEG_PRED120,SECOND_LEG_AIRLINE,SECOND_LEG_ORIG,SECOND_LEG_ORIG_CITY,SECOND_LEG_DEST,SECOND_LEG_DEST_CITY,SECOND_LEG_DATE,SECOND_LEG_DEP_TIME,SECOND_LEG_ARR_TIME,NEXT_BEST_SECOND_LEG_DATE,NEXT_BEST_SECOND_LEG_DEP_TIME,NEXT_BEST_SECOND_LEG_ARR_TIME,FIRST_LEG_ORIG_TZ,FIRST_LEG_DEST_TZ,SECOND_LEG_ORIG_TZ,SECOND_LEG_DEST_TZ,FIRST_LEG_DEP_TIMESTAMP,FIRST_LEG_ARR_TIMESTAMP,SECOND_LEG_DEP_TIMESTAMP,SECOND_LEG_ARR_TIMESTAMP,NEXT_BEST_SECOND_LEG_DEP_TIMESTAMP,NEXT_BEST_SECOND_LEG_ARR_TIMESTAMP,overnight_bool_1,overnight_bool_2,overnight_bool_3,overnight_connection,FIRST_FLIGHT_DURATION,SECOND_FLIGHT_DURATION,CONNECT_TIME,TRIP_TIME,RISK_MISSED_CONNECTION,NEXT_FLIGHT_TIMELOSS,TOTAL_RISK
0,UA,ATL,"Atlanta, GA",DEN,"Denver, CO",12/3/2019,700,829,0.045453,0.027641,0.037959,0.000000,0.027694,0.009809,0.009421,0.009879,UA,DEN,"Denver, CO",SEA,"Seattle, WA",12/3/2019,930,1134,12/3/2019,1100,1310,US/Eastern,US/Mountain,US/Mountain,US/Pacific,2019-12-03 07:00:00-05:00,2019-12-03 08:29:00-07:00,2019-12-03 09:30:00-07:00,2019-12-03 11:34:00-08:00,2019-12-03 11:00:00-07:00,2019-12-03 13:10:00-08:00,0,0,0,False,209.0,184.0,61.0,454.0,0.027641,96.0,2.653524
1,DL,ATL,"Atlanta, GA",PDX,"Portland, OR",12/3/2019,943,1223,0.223754,0.290545,0.296317,0.278417,0.335716,0.286920,0.257592,0.270189,DL,PDX,"Portland, OR",SEA,"Seattle, WA",12/3/2019,1310,1418,12/3/2019,1700,1812,US/Eastern,US/Pacific,US/Pacific,US/Pacific,2019-12-03 09:43:00-05:00,2019-12-03 12:23:00-08:00,2019-12-03 13:10:00-08:00,2019-12-03 14:18:00-08:00,2019-12-03 17:00:00-08:00,2019-12-03 18:12:00-08:00,0,0,0,False,340.0,68.0,47.0,455.0,0.223754,234.0,52.358382
2,UA,ATL,"Atlanta, GA",ORD,"Chicago, IL",12/3/2019,805,919,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,UA,ORD,"Chicago, IL",SEA,"Seattle, WA",12/3/2019,1015,1256,12/3/2019,1155,1437,US/Eastern,US/Central,US/Central,US/Pacific,2019-12-03 08:05:00-05:00,2019-12-03 09:19:00-06:00,2019-12-03 10:15:00-06:00,2019-12-03 12:56:00-08:00,2019-12-03 11:55:00-06:00,2019-12-03 14:37:00-08:00,0,0,0,False,134.0,281.0,56.0,471.0,0.000000,101.0,0.000000
3,AA,ATL,"Atlanta, GA",DFW,"Dallas/Fort Worth, TX",12/3/2019,1000,1136,0.133624,0.070994,0.127478,0.000000,0.000000,0.000000,0.000000,0.009646,AA,DFW,"Dallas/Fort Worth, TX",SEA,"Seattle, WA",12/3/2019,1226,1459,12/3/2019,1445,1713,US/Eastern,US/Central,US/Central,US/Pacific,2019-12-03 10:00:00-05:00,2019-12-03 11:36:00-06:00,2019-12-03 12:26:00-06:00,2019-12-03 14:59:00-08:00,2019-12-03 14:45:00-06:00,2019-12-03 17:13:00-08:00,0,0,0,False,156.0,273.0,50.0,479.0,0.133624,134.0,17.905566
4,UA,ATL,"Atlanta, GA",IAH,"Houston, TX",12/3/2019,730,853,0.478738,0.498231,0.530831,0.577974,0.639077,0.537524,0.466954,0.606257,UA,IAH,"Houston, TX",SEA,"Seattle, WA",12/3/2019,938,1230,12/3/2019,1421,1717,US/Eastern,US/Central,US/Central,US/Pacific,2019-12-03 07:30:00-05:00,2019-12-03 08:53:00-06:00,2019-12-03 09:38:00-06:00,2019-12-03 12:30:00-08:00,2019-12-03 14:21:00-06:00,2019-12-03 17:17:00-08:00,0,0,0,False,143.0,292.0,45.0,480.0,0.478738,287.0,137.397702
5,WN,ATL,"Atlanta, GA",DAL,"Dallas, TX",12/3/2019,700,815,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,WN,DAL,"Dallas, TX",SEA,"Seattle, WA",12/3/2019,935,1210,12/4/2019,935,1210,US/Eastern,US/Central,US/Central,US/Pacific,2019-12-03 07:00:00-05:00,2019-12-03 08:15:00-06:00,2019-12-03 09:35:00-06:00,2019-12-03 12:10:00-08:00,2019-12-04 09:35:00-06:00,2019-12-04 12:10:00-08:00,0,0,0,False,135.0,275.0,80.0,490.0,0.000000,1440.0,0.000000
6,AA,ATL,"Atlanta, GA",DFW,"Dallas/Fort Worth, TX",12/3/2019,753,929,0.429799,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,AA,DFW,"Dallas/Fort Worth, TX",SEA,"Seattle, WA",12/3/2019,1032,1305,12

In this example "arr_no_later" is set to the following day, so more results are presented.

In [4]:
origin = 'SAN'
destination = 'ANC'
flight_date = '12/3/2019'
arr_no_later_date = '12/4/2019'

dne = '1'
dnl = '2359'
ane = '1'
anl = '2359'
tc = 45

df = itineraryBuilder('faa_2019_12', origin, destination, flight_date, arr_no_later_date, tc, dne, dnl, ane, anl); df.head(20)

C:\Users\parso\OneDrive\Documents\Education\Data Analytics\CSE-6242_Data and Visual Analytics\Project\gatech-cse6242-spring2023-project\dataManagement\itineraryBuilder.py:108: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  df['FIRST_LEG_ARR_TIMESTAMP'] = df['FIRST_LEG_ARR_TIMESTAMP'] + df['overnight_bool_1'].astype('timedelta64[D]')


,FIRST_LEG_AIRLINE,FIRST_LEG_ORIG,FIRST_LEG_ORIG_CITY,FIRST_LEG_DEST,FIRST_LEG_DEST_CITY,FIRST_LEG_DATE,FIRST_LEG_DEP_TIME,FIRST_LEG_ARR_TIME,FIRST_LEG_PRED15,FIRST_LEG_PRED30,FIRST_LEG_PRED45,FIRST_LEG_PRED60,FIRST_LEG_PRED75,FIRST_LEG_PRED90,FIRST_LEG_PRED105,FIRST_LEG_PRED120,SECOND_LEG_AIRLINE,SECOND_LEG_ORIG,SECOND_LEG_ORIG_CITY,SECOND_LEG_DEST,SECOND_LEG_DEST_CITY,SECOND_LEG_DATE,SECOND_LEG_DEP_TIME,SECOND_LEG_ARR_TIME,NEXT_BEST_SECOND_LEG_DATE,NEXT_BEST_SECOND_LEG_DEP_TIME,NEXT_BEST_SECOND_LEG_ARR_TIME,FIRST_LEG_ORIG_TZ,FIRST_LEG_DEST_TZ,SECOND_LEG_ORIG_TZ,SECOND_LEG_DEST_TZ,FIRST_LEG_DEP_TIMESTAMP,FIRST_LEG_ARR_TIMESTAMP,SECOND_LEG_DEP_TIMESTAMP,SECOND_LEG_ARR_TIMESTAMP,NEXT_BEST_SECOND_LEG_DEP_TIMESTAMP,NEXT_BEST_SECOND_LEG_ARR_TIMESTAMP,overnight_bool_1,overnight_bool_2,overnight_bool_3,FIRST_FLIGHT_DURATION,SECOND_FLIGHT_DURATION,CONNECT_TIME,TRIP_TIME,RISK_MISSED_CONNECTION,NEXT_FLIGHT_TIMELOSS,TOTAL_RISK
0,AS,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,615,925,0.008699,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,AS,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1010,1300,12/3/2019,1135,1420,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-03 06:15:00-08:00,2019-12-03 09:25:00-08:00,2019-12-03 10:10:00-08:00,2019-12-03 13:00:00-09:00,2019-12-03 11:35:00-08:00,2019-12-03 14:20:00-09:00,0,0,0,190.0,230.0,45.0,465.0,0.008699,80.0,0.695886
1,AS,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,735,1045,0.757457,0.010000,0.008680,0.000000,0.000000,0.000000,0.000000,0.000000,AS,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1140,1430,12/3/2019,1355,1645,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-03 07:35:00-08:00,2019-12-03 10:45:00-08:00,2019-12-03 11:40:00-08:00,2019-12-03 14:30:00-09:00,2019-12-03 13:55:00-08:00,2019-12-03 16:45:00-09:00,0,0,0,190.0,230.0,55.0,475.0,0.757457,135.0,102.256756
2,DL,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,725,1036,0.015958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1135,1420,12/3/2019,1140,1430,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-03 07:25:00-08:00,2019-12-03 10:36:00-08:00,2019-12-03 11:35:00-08:00,2019-12-03 14:20:00-09:00,2019-12-03 11:40:00-08:00,2019-12-03 14:30:00-09:00,0,0,0,191.0,225.0,59.0,475.0,0.015958,10.0,0.159583
3,AS,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,1545,1849,0.003577,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,AS,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1955,2241,12/3/2019,2055,2340,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-03 15:45:00-08:00,2019-12-03 18:49:00-08:00,2019-12-03 19:55:00-08:00,2019-12-03 22:41:00-09:00,2019-12-03 20:55:00-08:00,2019-12-03 23:40:00-09:00,0,0,0,184.0,226.0,66.0,476.0,0.000000,59.0,0.000000
4,AS,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,940,1250,0.000000,0.029787,0.030988,0.033043,0.043365,0.030991,0.024204,0.009761,AS,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1355,1645,12/3/2019,1640,1926,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-03 09:40:00-08:00,2019-12-03 12:50:00-08:00,2019-12-03 13:55:00-08:00,2019-12-03 16:45:00-09:00,2019-12-03 16:40:00-08:00,2019-12-03 19:26:00-09:00,0,0,0,190.0,230.0,65.0,485.0,0.029787,161.0,4.795657
5,AS,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,1335,1645,0.040826,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,AS,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1755,2041,12/3/2019,1955,2241,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-03 13:35:00-08:00,2019-12-03 16:45:00-08:00,2019-12-03 17:55:00-08:00,2019-12-03 20:41:00-09:00,2019-12-03 19:55:00-08:00,2019-12-03 22:41:00-09:00,0,0,0,190.0,226.0,70.0,486.0,0.000000,120.0,0.000000
6,DL,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,1313,1629,0.044967,0.088820,0.048456,0.054901,0.058015,0.038084,0.057319,0.028724,DL,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1745,2034,12/3/2019,1755,2041,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-

You can also order by trip duration as shown. Other options include to sort by "earliest_arrival" and "min_connection_time".

In [7]:
df = itineraryBuilder('faa_2019_12', origin, destination, flight_date, arr_no_later_date, tc, dne, dnl, ane, anl, orderby='duration'); df.head(10)

C:\Users\parso\OneDrive\Documents\Education\Data Analytics\CSE-6242_Data and Visual Analytics\Project\gatech-cse6242-spring2023-project\dataManagement\itineraryBuilder.py:108: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  df['FIRST_LEG_ARR_TIMESTAMP'] = df['FIRST_LEG_ARR_TIMESTAMP'] + df['overnight_bool_1'].astype('timedelta64[D]')


,FIRST_LEG_AIRLINE,FIRST_LEG_ORIG,FIRST_LEG_ORIG_CITY,FIRST_LEG_DEST,FIRST_LEG_DEST_CITY,FIRST_LEG_DATE,FIRST_LEG_DEP_TIME,FIRST_LEG_ARR_TIME,FIRST_LEG_PRED15,FIRST_LEG_PRED30,FIRST_LEG_PRED45,FIRST_LEG_PRED60,FIRST_LEG_PRED75,FIRST_LEG_PRED90,FIRST_LEG_PRED105,FIRST_LEG_PRED120,SECOND_LEG_AIRLINE,SECOND_LEG_ORIG,SECOND_LEG_ORIG_CITY,SECOND_LEG_DEST,SECOND_LEG_DEST_CITY,SECOND_LEG_DATE,SECOND_LEG_DEP_TIME,SECOND_LEG_ARR_TIME,NEXT_BEST_SECOND_LEG_DATE,NEXT_BEST_SECOND_LEG_DEP_TIME,NEXT_BEST_SECOND_LEG_ARR_TIME,FIRST_LEG_ORIG_TZ,FIRST_LEG_DEST_TZ,SECOND_LEG_ORIG_TZ,SECOND_LEG_DEST_TZ,FIRST_LEG_DEP_TIMESTAMP,FIRST_LEG_ARR_TIMESTAMP,SECOND_LEG_DEP_TIMESTAMP,SECOND_LEG_ARR_TIMESTAMP,NEXT_BEST_SECOND_LEG_DEP_TIMESTAMP,NEXT_BEST_SECOND_LEG_ARR_TIMESTAMP,overnight_bool_1,overnight_bool_2,overnight_bool_3,FIRST_FLIGHT_DURATION,SECOND_FLIGHT_DURATION,CONNECT_TIME,TRIP_TIME,RISK_MISSED_CONNECTION,NEXT_FLIGHT_TIMELOSS,TOTAL_RISK
0,AS,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,615,925,0.008699,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,AS,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1010,1300,12/3/2019,1135,1420,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-03 06:15:00-08:00,2019-12-03 09:25:00-08:00,2019-12-03 10:10:00-08:00,2019-12-03 13:00:00-09:00,2019-12-03 11:35:00-08:00,2019-12-03 14:20:00-09:00,0,0,0,190.0,230.0,45.0,465.0,0.008699,80.0,0.695886
1,AS,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,735,1045,0.757457,0.010000,0.008680,0.000000,0.000000,0.000000,0.000000,0.000000,AS,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1140,1430,12/3/2019,1355,1645,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-03 07:35:00-08:00,2019-12-03 10:45:00-08:00,2019-12-03 11:40:00-08:00,2019-12-03 14:30:00-09:00,2019-12-03 13:55:00-08:00,2019-12-03 16:45:00-09:00,0,0,0,190.0,230.0,55.0,475.0,0.757457,135.0,102.256756
2,DL,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,725,1036,0.015958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1135,1420,12/3/2019,1140,1430,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-03 07:25:00-08:00,2019-12-03 10:36:00-08:00,2019-12-03 11:35:00-08:00,2019-12-03 14:20:00-09:00,2019-12-03 11:40:00-08:00,2019-12-03 14:30:00-09:00,0,0,0,191.0,225.0,59.0,475.0,0.015958,10.0,0.159583
3,AS,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,1545,1849,0.003577,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,AS,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1955,2241,12/3/2019,2055,2340,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-03 15:45:00-08:00,2019-12-03 18:49:00-08:00,2019-12-03 19:55:00-08:00,2019-12-03 22:41:00-09:00,2019-12-03 20:55:00-08:00,2019-12-03 23:40:00-09:00,0,0,0,184.0,226.0,66.0,476.0,0.000000,59.0,0.000000
4,AS,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,940,1250,0.000000,0.029787,0.030988,0.033043,0.043365,0.030991,0.024204,0.009761,AS,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1355,1645,12/3/2019,1640,1926,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-03 09:40:00-08:00,2019-12-03 12:50:00-08:00,2019-12-03 13:55:00-08:00,2019-12-03 16:45:00-09:00,2019-12-03 16:40:00-08:00,2019-12-03 19:26:00-09:00,0,0,0,190.0,230.0,65.0,485.0,0.029787,161.0,4.795657
5,AS,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,1335,1645,0.040826,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,AS,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1755,2041,12/3/2019,1955,2241,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-03 13:35:00-08:00,2019-12-03 16:45:00-08:00,2019-12-03 17:55:00-08:00,2019-12-03 20:41:00-09:00,2019-12-03 19:55:00-08:00,2019-12-03 22:41:00-09:00,0,0,0,190.0,226.0,70.0,486.0,0.000000,120.0,0.000000
6,DL,SAN,"San Diego, CA",SEA,"Seattle, WA",12/3/2019,1313,1629,0.044967,0.088820,0.048456,0.054901,0.058015,0.038084,0.057319,0.028724,DL,SEA,"Seattle, WA",ANC,"Anchorage, AK",12/3/2019,1745,2034,12/3/2019,1755,2041,US/Pacific,US/Pacific,US/Pacific,US/Alaska,2019-12-

### Query Flights Function Call

Query flights includes many of the same parameters as Itinerary Builder and is in fact called by Itinerary Builder. This returns the initial query with the dates and times as string values.

In [8]:
origin = 'ATL'
destination = 'SEA'
flight_date = '12/3/2019'

dne = '700'
dnl = '1000'
ane = '1'
anl = '2000'

df = queryFlights('faa_2019_12', origin, destination, flight_date, dne, dnl, ane, anl); df

,FIRST_LEG_AIRLINE,FIRST_LEG_ORIG,FIRST_LEG_ORIG_CITY,FIRST_LEG_DEST,FIRST_LEG_DEST_CITY,FIRST_LEG_DATE,FIRST_LEG_DEP_TIME,FIRST_LEG_ARR_TIME,FIRST_LEG_PRED15,FIRST_LEG_PRED30,FIRST_LEG_PRED45,FIRST_LEG_PRED60,FIRST_LEG_PRED75,FIRST_LEG_PRED90,FIRST_LEG_PRED105,FIRST_LEG_PRED120,SECOND_LEG_AIRLINE,SECOND_LEG_ORIG,SECOND_LEG_ORIG_CITY,SECOND_LEG_DEST,SECOND_LEG_DEST_CITY,SECOND_LEG_DATE,SECOND_LEG_DEP_TIME,SECOND_LEG_ARR_TIME,NEXT_BEST_SECOND_LEG_DATE,NEXT_BEST_SECOND_LEG_DEP_TIME,NEXT_BEST_SECOND_LEG_ARR_TIME
0,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",12/3/2019,943,1117,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,AUS,"Austin, TX",SEA,"Seattle, WA",12/3/2019,610,855,12/4/2019,610,855
1,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",12/3/2019,820,954,0.745920,0.457844,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,AUS,"Austin, TX",SEA,"Seattle, WA",12/3/2019,610,855,12/4/2019,610,855
2,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",12/3/2019,943,1117,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,AUS,"Austin, TX",SEA,"Seattle, WA",12/4/2019,610,855,12/5/2019,610,855
3,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",12/3/2019,820,954,0.745920,0.457844,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,AUS,"Austin, TX",SEA,"Seattle, WA",12/4/2019,610,855,12/5/2019,610,855
4,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",12/3/2019,943,1117,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,AUS,"Austin, TX",SEA,"Seattle, WA",12/5/2019,610,855,12/3/2019,715,1036
5,DL,ATL,"Atlanta, GA",BNA,"Nashville, TN",12/3/2019,924,933,0.121370,0.169947,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,BNA,"Nashville, TN",SEA,"Seattle, WA",12/3/2019,715,1036,12/4/2019,715,1036
6,DL,ATL,"Atlanta, GA",BNA,"Nashville, TN",12/3/2019,812,820,0.000000,0.015957,0.009164,0.000000,0.000000,0.000000,0.000000,0.000000,DL,BNA,"Nashville, TN",SEA,"Seattle, WA",12/3/2019,715,1036,12/4/2019,715,1036
7,DL,ATL,"Atlanta, GA",BNA,"Nashville, TN",12/3/2019,924,933,0.121370,0.169947,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,BNA,"Nashville, TN",SEA,"Seattle, WA",12/4/2019,715,1036,12/5/2019,715,1036
8,DL,ATL,"Atlanta, GA",BNA,"Nashville, TN",12/3/2019,812,820,0.000000,0.015957,0.009164,0.000000,0.000000,0.000000,0.000000,0.000000,DL,BNA,"Nashville, TN",SEA,"Seattle, WA",12/4/2019,715,1036,12/5/2019,715,1036
9,DL,ATL,"Atlanta, GA",BNA,"Nashville, TN",12/3/2019,924,933,0.121370,0.169947,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DL,BNA,"Nashville, TN",SEA,"Seattle, WA",12/5/2019,715,1036,12/3/2019,640,1024


### getValidDestinations function call

Use this function to get a list of locations which can be reached with exactly two flights from the origin city, within a two day period.

In [10]:
df2 = getValidDestinations('faa_2019_12', origin, flight_date); df2

,AIRPORT,CITY
0,ABR,"Aberdeen, SD"
1,BQN,"Aguadilla, PR"
2,CAK,"Akron, OH"
3,ABY,"Albany, GA"
4,ALB,"Albany, NY"
5,ABQ,"Albuquerque, NM"
6,AEX,"Alexandria, LA"
7,ABE,"Allentown/Bethlehem/Easton, PA"
8,APN,"Alpena, MI"
9,AMA,"Amarillo, TX"
